# PROJETO 1
 Aluna: Dara Maria Barbosa de Sousa

#### O codigo a seguir está estruturado conforme o exemplo de um tutorial que pode ser encontrado nesse link: https://machinelearningmastery.com/machine-learning-in-python-step-by-step/

# Importando bibliotecas

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Preparando os dados

In [3]:
dataset = pd.read_csv('database.csv')
dataset.head()

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1,77.0,1,2.0,1.0,1.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0


Para um melhor desempenho do modelo as células vazias serão substituidas pelo valor 0.

In [4]:
dataset = dataset.fillna(0)
dataset.head()

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1,77.0,1,2.0,1.0,1.0,2.0,0.0,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,0.0,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0


## 1. Admissão ao hospital

### Preparando os dados para a condiçao estabelecida:
O tempo de admissão no hospital: todas as colunas de entrada (2-112) exceto 93, 94, 95, 100, 101, 102, 103, 104, 105 podem ser usadas para previsão.

In [5]:
#Dataframe somente com as colunas indicadas na descrição acima.
df1 = pd.DataFrame(dataset[dataset.columns[2:93]])
df1 = df1.join(dataset[dataset.columns[96:100]])
df1 = df1.join(dataset[dataset.columns[106:113]])
df1.head()

,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,LID_KB,NITR_S,NA_R_1_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS
0,1,2.0,1.0,1.0,2.0,0.0,3.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0
2,1,0.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
3,0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
4,1,0.0,0.0,0.0,2.0,0.0,3.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0


In [6]:
# Divisão do dados em 70% para treino e 30% teste
array = df1.values
X = array[:,0:101]
y = array[:,101]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.30, random_state=1)

In [7]:
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))

# Avaliar cada modelo por vez
results = []
names = []
for name, model in models:
 kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
 results.append(cv_results)
 names.append(name)
 print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LDA: 0.894958 (0.011430)
QDA: 0.237815 (0.038150)


/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:

KNN: 0.900840 (0.009800)
CART: 0.823529 (0.035652)


### Predição

In [8]:
# Predição com conjuto de dados de validação
model = KNeighborsClassifier()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)


# Avaliando a predição
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8803921568627451
[[448   5]
 [ 56   1]]
              precision    recall  f1-score   support

         0.0       0.89      0.99      0.94       453
         1.0       0.17      0.02      0.03        57

    accuracy                           0.88       510
   macro avg       0.53      0.50      0.48       510
weighted avg       0.81      0.88      0.84       510



## 2. Terceiro dia

### Preparando os dados para a condiçao estabelecida:
No final do terceiro dia (72 horas após a admissão no hospital), todas as colunas de entrada (2-112) podem ser usadas para previsão.

In [9]:
#Dataframe somente com as colunas indicadas na descrição acima.
df2 = pd.DataFrame(dataset[dataset.columns[2:113]])
df2.head()

,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS
0,1,2.0,1.0,1.0,2.0,0.0,3.0,0.0,7.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0
2,1,0.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,...,2.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
3,0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
4,1,0.0,0.0,0.0,2.0,0.0,3.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0


In [10]:
# Divisão do dados em 70% para treino e 30% teste
array = df2.values
X = array[:,0:110]
y = array[:,110]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.30, random_state=1)

In [11]:
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))

# Avaliar cada modelo por vez
results = []
names = []
for name, model in models:
 kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
 results.append(cv_results)
 names.append(name)
 print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LDA: 0.890756 (0.013550)
QDA: 0.259664 (0.037099)


/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/daramaria/Documentos/UNB/F. SISTEMAS INTELIGENTES/TUTORIAL/myenv/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:

KNN: 0.900840 (0.009800)
CART: 0.821008 (0.031454)


### Predição

In [14]:
# Predição com conjuto de dados de validação
model = KNeighborsClassifier()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)


# Avaliando a predição
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8823529411764706
[[449   4]
 [ 56   1]]
              precision    recall  f1-score   support

         0.0       0.89      0.99      0.94       453
         1.0       0.20      0.02      0.03        57

    accuracy                           0.88       510
   macro avg       0.54      0.50      0.48       510
weighted avg       0.81      0.88      0.84       510



### Conclusão
Podemos concluir que os modelos em geral possuem uma boa acuracia para o conjunto de dados, sendo o K-nearest neighbors com um resultado um pouco melhor que os outros.

### Referencias:
- https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
- https://www.w3schools.com/python/pandas/pandas_analyzing.asp
- https://estatsite.com.br/2020/07/25/dividindo-o-dataset-em-treino-e-teste-no-python/
- https://machinelearningmastery.com/index-slice-reshape-numpy-arrays-machine-learning-python/
- https://ateliware.com/blog/classificacao-knn-k-means
- https://scikit-learn.org/stable/modules/lda_qda.html